In [42]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util, CrossEncoder

df = pd.read_csv('pycon_2024_sessions.csv')

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-mpnet-base-v2')

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')
# Compute embeddings for all descriptions
embeddings = model.encode(df['Description'].tolist(), convert_to_tensor=True)

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [77]:
# current_speech = df.loc[df['Title'] == 'Embeddings: How Computers Learned to Read—PyCon AU 2024'].index[0]
current_speech = 0

In [78]:
related_articles = util.semantic_search(embeddings, embeddings, top_k=11)

In [79]:
for item in related_articles[0]:
    print(item['corpus_id'])

0
56
15
29
37
6
36
47
7
52
34


In [80]:
df['Description'][16]

"As large language models take over the world, we’re now working alongside machines that can read, write and converse – coding with CoPilot, chatting with ChatGPT and drawing with DALL-E. But how do machines, which fundamentally operate on binary code, achieve such remarkable feats? The answer lies in embeddings. Embeddings allow us to represent complex data - whether it's text, images, or even abstract concepts - as dense vectors of numbers. In this presentation, we'll demystify embeddings and give you a practical and intuitive understanding of how they work."

In [81]:
related_articles[0]

[{'corpus_id': 0, 'score': 1.0000001192092896},
 {'corpus_id': 56, 'score': 0.49006807804107666},
 {'corpus_id': 15, 'score': 0.4833514094352722},
 {'corpus_id': 29, 'score': 0.4181976318359375},
 {'corpus_id': 37, 'score': 0.38466009497642517},
 {'corpus_id': 6, 'score': 0.3662496507167816},
 {'corpus_id': 36, 'score': 0.3275807797908783},
 {'corpus_id': 47, 'score': 0.32603365182876587},
 {'corpus_id': 7, 'score': 0.32285377383232117},
 {'corpus_id': 52, 'score': 0.32064878940582275},
 {'corpus_id': 34, 'score': 0.3084678649902344}]

In [82]:
ids = [item['corpus_id'] for item in related_articles[0]]

In [83]:
ids

[0, 56, 15, 29, 37, 6, 36, 47, 7, 52, 34]

In [84]:
cross_inp = [[df.iloc[current_speech]['Description'], df.iloc[id]['Description']] for id in ids]

In [85]:
cross_inp

[["Sick of writing lots of dedicated methods to perform the same tasks? Want to write less code? This talk may be for you! \nREST API's are ubiquitious, and a very useful way to send and recieve data from various services. However, building them can often feel repetitive and boring. So, because I was lazy, and sick of doing it I decided to override specific dunder methods to write less boiler plate code, and keep myself interested.\nWas it easier? Probably not. Did it require less work than a normal implementation? Also probably not. Did it make my code more readable and portable? Definitely not. But it was fun, and I learnt how overriding Python's magic methods - often called dunders - could be useful. Let me share the lessons I learnt with you and inspire your own ideas.",
  "Sick of writing lots of dedicated methods to perform the same tasks? Want to write less code? This talk may be for you! \nREST API's are ubiquitious, and a very useful way to send and recieve data from various s

In [86]:
cross_scores = cross_encoder.predict(cross_inp)

In [87]:
cross_scores

array([ 6.2196136, -4.108149 , -3.261598 , -3.4519835, -2.692816 ,
       -3.2781358, -3.8076372, -2.0923877, -3.176807 , -4.3924775,
       -4.4125705], dtype=float32)

In [88]:
# Sort results by the cross-encoder scores
for idx in range(len(cross_scores)):
    related_articles[current_speech][idx]['cross-score'] = cross_scores[idx]

In [89]:
related_articles[current_speech]

[{'corpus_id': 0, 'score': 1.0000001192092896, 'cross-score': 6.2196136},
 {'corpus_id': 56, 'score': 0.49006807804107666, 'cross-score': -4.108149},
 {'corpus_id': 15, 'score': 0.4833514094352722, 'cross-score': -3.261598},
 {'corpus_id': 29, 'score': 0.4181976318359375, 'cross-score': -3.4519835},
 {'corpus_id': 37, 'score': 0.38466009497642517, 'cross-score': -2.692816},
 {'corpus_id': 6, 'score': 0.3662496507167816, 'cross-score': -3.2781358},
 {'corpus_id': 36, 'score': 0.3275807797908783, 'cross-score': -3.8076372},
 {'corpus_id': 47, 'score': 0.32603365182876587, 'cross-score': -2.0923877},
 {'corpus_id': 7, 'score': 0.32285377383232117, 'cross-score': -3.176807},
 {'corpus_id': 52, 'score': 0.32064878940582275, 'cross-score': -4.3924775},
 {'corpus_id': 34, 'score': 0.3084678649902344, 'cross-score': -4.4125705}]

In [90]:
sorted(related_articles[current_speech], key=lambda x: x['cross-score'], reverse=True)

[{'corpus_id': 0, 'score': 1.0000001192092896, 'cross-score': 6.2196136},
 {'corpus_id': 47, 'score': 0.32603365182876587, 'cross-score': -2.0923877},
 {'corpus_id': 37, 'score': 0.38466009497642517, 'cross-score': -2.692816},
 {'corpus_id': 7, 'score': 0.32285377383232117, 'cross-score': -3.176807},
 {'corpus_id': 15, 'score': 0.4833514094352722, 'cross-score': -3.261598},
 {'corpus_id': 6, 'score': 0.3662496507167816, 'cross-score': -3.2781358},
 {'corpus_id': 29, 'score': 0.4181976318359375, 'cross-score': -3.4519835},
 {'corpus_id': 36, 'score': 0.3275807797908783, 'cross-score': -3.8076372},
 {'corpus_id': 56, 'score': 0.49006807804107666, 'cross-score': -4.108149},
 {'corpus_id': 52, 'score': 0.32064878940582275, 'cross-score': -4.3924775},
 {'corpus_id': 34, 'score': 0.3084678649902344, 'cross-score': -4.4125705}]

In [91]:
for item in related_articles[current_speech]:
    print(f"Title: {df.iloc[item['corpus_id']]['Title']} - Cross-encoder Score: {item['score']}")

Title: A Lazy Person's Guide to Building REST Clients or: How I Learned to Stop Worrying and Love Dunder Overrides—PyCon AU 2024 - Cross-encoder Score: 1.0000001192092896
Title: What Python Can Learn From Other Languages—PyCon AU 2024 - Cross-encoder Score: 0.49006807804107666
Title: Django, REST and OpenAPI - a gentle introduction—PyCon AU 2024 - Cross-encoder Score: 0.4833514094352722
Title: Let's make a working implementation of async functions in Python 2.1, also, why you might want to use a more recent version of Python—PyCon AU 2024 - Cross-encoder Score: 0.4181976318359375
Title: Performant Python—PyCon AU 2024 - Cross-encoder Score: 0.38466009497642517
Title: Better dataframes—PyCon AU 2024 - Cross-encoder Score: 0.3662496507167816
Title: Notes on Over-Engineering: A Project Post-Mortem—PyCon AU 2024 - Cross-encoder Score: 0.3275807797908783
Title: The perfect setup? Not setup.py! Building packages the right way—PyCon AU 2024 - Cross-encoder Score: 0.32603365182876587
Title: Bu

In [92]:
for item in sorted(related_articles[current_speech], key=lambda x: x['cross-score'], reverse=True):
    print(f"Title: {df.iloc[item['corpus_id']]['Title']} - Cross-encoder Score: {item['cross-score']}")

Title: A Lazy Person's Guide to Building REST Clients or: How I Learned to Stop Worrying and Love Dunder Overrides—PyCon AU 2024 - Cross-encoder Score: 6.219613552093506
Title: The perfect setup? Not setup.py! Building packages the right way—PyCon AU 2024 - Cross-encoder Score: -2.0923876762390137
Title: Performant Python—PyCon AU 2024 - Cross-encoder Score: -2.6928160190582275
Title: Build Your Own IoT! Fun With Python and Raspberry Pi.—PyCon AU 2024 - Cross-encoder Score: -3.176806926727295
Title: Django, REST and OpenAPI - a gentle introduction—PyCon AU 2024 - Cross-encoder Score: -3.2615981101989746
Title: Better dataframes—PyCon AU 2024 - Cross-encoder Score: -3.2781357765197754
Title: Let's make a working implementation of async functions in Python 2.1, also, why you might want to use a more recent version of Python—PyCon AU 2024 - Cross-encoder Score: -3.4519834518432617
Title: Notes on Over-Engineering: A Project Post-Mortem—PyCon AU 2024 - Cross-encoder Score: -3.8076372146606

In [21]:
temp = df.iloc[ids]

In [ ]:
temp

In [1]:
import streamlit as st
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch

# Load your data
df = pd.read_csv('pycon_2024_sessions.csv')  # Assuming you have the DataFrame from the previous step

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-mpnet-base-v2')

# Compute embeddings for all descriptions
embeddings = model.encode(df['Description'].tolist(), convert_to_tensor=True)
df['embedding'] = embeddings.tolist()

related_articles = util.semantic_search(embeddings, embeddings, top_k=11)

# Streamlit UI to select a title
selected_title = st.selectbox("Select a session title", df['Title'].values)

# Get the selected description
# selected_description = df[df['Title'] == selected_title]['Description'].values[0]

# get selected id
selected_id = df[df['Title'] == selected_title].index[0]

related_ids = [item['corpus_id'] for item in related_articles[selected_id][0]]

# Compute embedding for the selected description
# selected_embedding = model.encode(selected_description, convert_to_tensor=True)

# Compute cosine similarities between the selected description and all others
# cosine_similarities = util.pytorch_cos_sim(selected_embedding, embeddings).flatten()

# Find the top N most similar descriptions
N = st.slider("Number of similar sessions to display", 1, 10, 5)
top_n_indices = related_ids[1:N + 1]  # Skip the first as it's the selected one

# Display the most similar descriptions
st.write(f"Top {N} most similar sessions to '{selected_title}':")
for index in top_n_indices:
    index = int(index)  # Convert tensor index to int
    st.write(f"**{df.iloc[index]['Title']}**")
    st.write(df.iloc[index]['Description'])
    st.write("----")


ModuleNotFoundError: No module named 'streamlit'